In [1]:
import numpy as np
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torch.nn.functional as F

import time, pathlib, os, sys

## 2D dataset

In [2]:
class LipschitzNet(nn.Module):
    
    def __init__(self, net, K=1., lamda=1.):
        super().__init__()
        
        self.net = net
        self.lamda = lamda
        self.X = None
        
        self.Y = None
        self.dydx = None
        self.K = K
        
        self.gp = 0
        self.gclipper = torch.Tensor([999])
        
    def forward(self, x):
        self.X = torch.autograd.Variable(x, requires_grad=True)
        
        self.Y = self.net(self.X)
        y = self.Y+0.
        y.register_hook(self.scale_gradient_back)
        return y
    
    def scale_gradient_back(self, grad):
#         print("Old", grad.shape)
#         grad = torch.min(torch.max(grad, -self.gclipper), self.gclipper)
#         print("New", grad.shape)
#         return grad
#         return torch.min(torch.max(grad, -self.gclipper), self.gclipper)
        return torch.minimum(torch.maximum(grad, -self.gclipper), self.gclipper)

    
    def get_dydx(self):
        self.dydx = torch.autograd.grad(outputs=self.Y, inputs=self.X,
                                    grad_outputs=torch.ones_like(self.Y),
                                    only_inputs=True, retain_graph=True, create_graph=True)[0]
        return self.dydx
    
    def get_gradient_penalty(self):
#         m = self.dydx.shape[0]
        dydx_norm = torch.norm(self.dydx, p=2, dim=1, keepdim=True)
#         self.cond = -(torch.abs(dydx_norm/self.K) -1.)
        self.cond = -dydx_norm/self.K +1.
        
#         a=-20
#         intolerables = torch.log(torch.exp(a*(self.cond-0.1))+1)/a
#         dydx_norm = torch.norm(self.dydx.data, p=2, dim=1, keepdim=True)
#         self.gp = 0.5*((intolerables*5)**2).mean()*self.lamda

#         intolerables = torch.clamp(F.softplus(self.cond-0.1, beta=-20), -1, 1)
        intolerables = F.softplus(self.cond-0.1, beta=-20)
#         intolerables = F.softplus(F.softplus(self.cond-0.1, beta=-20)+2, beta=5)-2

        self.gp = (self.smooth_l1(intolerables*5)).mean()*self.lamda
        
    
#         self.gp = 0.5*(intolerables**2).sum()*self.lamda
        
#         intolerables = torch.min(self.cond-1e-1, torch.zeros_like(self.cond))
#         self.gp = 0.5*(intolerables**2).mean()*self.lamda
#         self.gp = torch.abs(intolerables).mean()*self.lamda
#         self.gp = 0.5*(torch.abs(intolerables)+intolerables**2).mean()*self.lamda

        return self.gp
    
    ##
    def get_gradient_clipper(self):
        with torch.no_grad():
            cond = self.cond.data
            linear_mask = cond>0.14845
#             print(dydx.shape, linear_mask.shape)
            a = 20.
            gclipper = -((1.05*(cond-1))**4)+1
            gclipper = torch.log(torch.exp(a*gclipper)+1)/a
#             print(gclipper.shape)
            gc2 = 3*cond-0.0844560006
#             print(gc2.shape)
            gclipper[linear_mask] = gc2[linear_mask]
#             print(gclipper.shape)
#             gclipper = torch.clamp(gclipper, min=0.01)
            self.gclipper = gclipper
            
        return self.gclipper

    def smooth_l1(self, x, beta=1):
        mask = x<beta
        y = torch.empty_like(x)
        y[mask] = 0.5*(x[mask]**2)/beta
        y[~mask] = torch.abs(x[~mask])-0.5*beta
        return y
    
    def compute_penalty_and_clipper(self):
        self.get_dydx()
        self.get_gradient_penalty()
        self.get_gradient_clipper()
        return

In [3]:
class GradientPenaltyNet(nn.Module):
    
    def __init__(self, net, K=1., lamda=1.):
        super().__init__()
        
        self.net = net
        self.lamda = lamda
        self.X = None
        
        self.Y = None
        self.dydx = None
        self.K = K
        
        self.gp = 0
        
    def forward(self, x):
        self.X = torch.autograd.Variable(x, requires_grad=True)
        
        self.Y = self.net(self.X)
        y = self.Y+0.
        return y

    def get_dydx(self):
        self.dydx = torch.autograd.grad(outputs=self.Y, inputs=self.X,
                                    grad_outputs=torch.ones_like(self.Y),
                                    only_inputs=True, retain_graph=True, create_graph=True)[0]
        return self.dydx
    
    def get_gradient_penalty(self):
        dydx_norm = torch.norm(self.dydx, p=2, dim=1, keepdim=True)
        
        self.gp = ((dydx_norm-1)**2).mean()*self.lamda
        return self.gp
    
    def MSE_loss(self, diff):
        return 0.5*(diff**2).mean()
    
    def compute_penalty(self):
        self.get_dydx()
        self.get_gradient_penalty()
        return

In [4]:
class LipschitzPenaltyNet(nn.Module):
    
    def __init__(self, net, K=1., lamda=1.):
        super().__init__()
        
        self.net = net
        self.lamda = lamda
        self.X = None
        
        self.Y = None
        self.dydx = None
        self.K = K
        
        self.gp = 0
        
    def forward(self, x):
        self.X = torch.autograd.Variable(x, requires_grad=True)
        
        self.Y = self.net(self.X)
        y = self.Y+0.
        return y

    def get_dydx(self):
        self.dydx = torch.autograd.grad(outputs=self.Y, inputs=self.X,
                                    grad_outputs=torch.ones_like(self.Y),
                                    only_inputs=True, retain_graph=True, create_graph=True)[0]
        return self.dydx
    
    def get_gradient_penalty(self):
        dydx_norm = torch.norm(self.dydx, p=2, dim=1, keepdim=True)
        
        self.gp = ((torch.maximum(dydx_norm-1, torch.Tensor([0])))**2).mean()*self.lamda
#         self.gp = ((dydx_norm-1)**2).mean()*self.lamda
        return self.gp
    
    def MSE_loss(self, diff):
        return 0.5*(diff**2).mean()
    
    def compute_penalty(self):
        self.get_dydx()
        self.get_gradient_penalty()
        return

In [5]:
log_file = './lipschitz_out/LOG_grads_constraint.txt'

In [6]:
%matplotlib tk

for data_indx in range(3):
    if data_indx == 1:
        # The two-dimensional domain of the fit.....
        ########https://scipython.com/blog/non-linear-least-squares-fitting-of-a-two-dimensional-data/#########
        x1min, x1max, nx1 = -5, 6, 75
        x2min, x2max, nx2 = -3, 7, 75
        x1, x2 = np.linspace(x1min, x1max, nx1), np.linspace(x2min, x2max, nx2)
        X1, X2 = np.meshgrid(x1, x2)

        # Our function to fit is going to be a sum of two-dimensional Gaussians
        def gaussian(x1, x2, x10, x20, x1alpha, x2alpha, A):
            return A * np.exp( -((x1-x10)/x1alpha)**2 -((x2-x20)/x2alpha)**2)

        # A list of the Gaussian parameters: x10, x20, x1alpha, x2alpha, A
        gprms = [(0, 2, 2.5, 5.4, 1.5),
                 (-1, 4, 6, 2.5, 1.8),
                 (-3, -0.5, 1, 2, 4),
                 (3, 0.5, 2, 1, 5)
                ]

        # Standard deviation of normally-distributed noise to add in generating
        # our test function to fit.
        # The function to be fit is Z.
        Y = np.zeros(X1.shape)
        for p in gprms:
            Y += gaussian(X1, X2, *p)
        ### Adding noise to the data
        # noise_sigma = 0.1
        # Z += noise_sigma * np.random.randn(*Z.shape)

        ####Scaling the data to range -1,1
        X1 = 2*(X1 - X1.min())/(X1.max() - X1.min()) -1
        X2 = 2*(X2 - X2.min())/(X2.max() - X2.min()) -1
        Y = 2*(Y - Y.min())/(Y.max() - Y.min()) -1
        Y = Y/2

        x1 = X1.reshape(-1)
        x2 = X2.reshape(-1)

        xx = torch.Tensor(np.c_[x1, x2])
        yy = torch.Tensor(Y.reshape(-1,1))
        
        
        ###########draw
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.plot_surface(X1, X2, Y, cmap='viridis')
        ax.set_xlabel('X1')
        ax.set_ylabel('X2')
        ax.set_zlabel('Y')
        # plt.show()
        plt.savefig("./lipschitz_out/data_regression_2.pdf", bbox_inches='tight')
        plt.close()
        
        plt.figure(figsize=(6/6*5,6.2/6*5))
        plt.axis('equal')
        lvls = 12
        plt.contourf(X1, X2, Y, levels=lvls)#, cmap=matplotlib.cm.bwr)

        clrs = ['white','white','white','white','white','white','white','k','k','k','k','k']
        cs = plt.contour(X1, X2, Y, levels=lvls, linestyles="None", colors="k", linewidths=1)
        plt.clabel(cs, cs.levels, inline=True, fontsize=10, fmt="%1.1f", colors=clrs)

        plt.locator_params(axis='x', nbins=5)
        plt.locator_params(axis='y', nbins=5)
        plt.xlabel("x1")
        plt.ylabel("x2")
        plt.xlim(-1,1)
        plt.ylim(-1,1)

        plt.savefig("./lipschitz_out/data_regression_2c.pdf", bbox_inches='tight')
        plt.close()
        
    elif data_indx == 0:
        num_points = 50
        X1 = np.linspace(-2.5, 1.5, num_points)
        # X1 = np.linspace(-2.5, 0, num_points)
        X2 = np.linspace(-2, 4, num_points)
        # X2 = np.linspace(-2, 2, num_points)
        X1, X2 = np.meshgrid(X1, X2)
        Y = np.sin(np.sqrt(X1**2 + X2**2))*2-1.

        ####Scaling the data to range -1,1
        X1 = 2*(X1 - X1.min())/(X1.max() - X1.min()) -1
        X2 = 2*(X2 - X2.min())/(X2.max() - X2.min()) -1
        Y = 2*(Y - Y.min())/(Y.max() - Y.min()) -1
        Y = Y

        x1 = X1.reshape(-1)
        x2 = X2.reshape(-1)

        xx = torch.Tensor(np.c_[x1, x2])
        yy = torch.Tensor(Y.reshape(-1,1))
        
        ###########draw
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.plot_surface(X1, X2, Y, cmap='viridis')
        ax.set_xlabel('X1')
        ax.set_ylabel('X2')
        ax.set_zlabel('Y')
        # plt.show()
        plt.savefig("./lipschitz_out/data_regression_1.pdf", bbox_inches='tight')
        plt.close()
        
        plt.figure(figsize=(6/6*5,6.2/6*5))
        plt.axis('equal')
        lvls = 12
        plt.contourf(X1, X2, Y, levels=lvls)#, cmap=matplotlib.cm.bwr)

        clrs = ['white','white','white','white','white','white','white','k','k','k','k','k']
        cs = plt.contour(X1, X2, Y, levels=lvls, linestyles="None", colors="k", linewidths=1)
        plt.clabel(cs, cs.levels, inline=True, fontsize=10, fmt="%1.1f", colors=clrs)

        plt.locator_params(axis='x', nbins=5)
        plt.locator_params(axis='y', nbins=5)
        plt.xlabel("x1")
        plt.ylabel("x2")
        plt.xlim(-1,1)
        plt.ylim(-1,1)

        plt.savefig("./lipschitz_out/data_regression_1c.pdf", bbox_inches='tight')
        plt.close()

    elif data_indx == 2:
        def twospirals(n_points, noise=.5 , ang=720):
            """
             Returns the two spirals dataset.
            """
            n = np.sqrt(np.random.rand(n_points,1)) * ang * (2*np.pi)/360
            d1x = -np.cos(n)*n + np.random.rand(n_points,1) * noise
            d1y = np.sin(n)*n + np.random.rand(n_points,1) * noise
            return (np.vstack((np.hstack((d1x,d1y)),np.hstack((-d1x,-d1y)))), 
                    np.hstack((np.zeros(n_points),np.ones(n_points))))

        np.random.seed(987)
        # x, y = twospirals(200, ang=420)
        x, y = twospirals(200, ang=400)
        x, Y = x/10, y.reshape(-1)
        X1, X2 = x[:,0], x[:,1]
        
        ####Scaling the data to range -1,1
        X1 = 2*(X1 - X1.min())/(X1.max() - X1.min()) -1
        X2 = 2*(X2 - X2.min())/(X2.max() - X2.min()) -1

        x1 = X1.reshape(-1)
        x2 = X2.reshape(-1)

        xx = torch.Tensor(np.c_[x1, x2])
        yy = torch.Tensor(Y.reshape(-1,1))
        
        ########## draw
        plt.scatter(x1, x2, c=y, s=50, edgecolors='k', lw=0.5)
        plt.grid()
        plt.savefig("./lipschitz_out/data_classification_1_invex.pdf", bbox_inches='tight')
        plt.close()
        
    else:
        raise NotImplementedError("Not implemented Error")
        
        
    
    network_seeds = [147, 258, 369]
    learning_rate = 0.005 #0.01 0.005
    EPOCHS = 7500 #5000 #7500

    

    if data_indx == 2:
        use_sigmoid = True
        criterion = nn.BCELoss()
        actf = nn.LeakyReLU
        
        class Sigmoid4(nn.Module):
            def forward(self,x):
                return torch.sigmoid(x*4)

        nn.Sigmoid = Sigmoid4
    else:
        criterion = nn.MSELoss()
        use_sigmoid = False
        actf = nn.ELU

    
    for lambda_ in [1, 3]:
        
        original_stdout = sys.stdout
        sys.stdout = open(log_file, 'a')
        print("===================================================")
        print(f"Dataset {data_indx}; lambda_={lambda_}")
        print("===================================================")
            # Reset the standard output
        sys.stdout.close()
        sys.stdout = original_stdout 

        #################################################
        per_step_time = []
        info_per_seed = []

        fig = plt.figure(figsize=(15,6))
        ax = fig.add_subplot(121,projection='3d')
        ax2 = fig.add_subplot(122)
        # ax.view_init(28,20)

        for ns in network_seeds:
            torch.manual_seed(ns)

            net_lips = nn.Sequential(nn.Linear(2,10),
                                     actf(),
                                     nn.Linear(10,10),
                                     actf(),
                                     nn.Linear(10,1),
                                     nn.Sigmoid() if use_sigmoid else nn.Identity())

            lipsNet = LipschitzNet(net_lips, K=1, lamda=lambda_)


            optimizer = torch.optim.Adam(lipsNet.parameters(), lr=learning_rate)

            for epoch in range(EPOCHS):
                start = time.time()

                yout = lipsNet(xx)    
                lipsNet.compute_penalty_and_clipper()
                loss = criterion(yout, yy) + lipsNet.gp

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                per_step_time.append(time.time()-start)

                if epoch%100 == 0:
                    min_val = float(lipsNet.cond.min())
                    max_k = float(torch.norm(lipsNet.dydx, p=2, dim=1, keepdim=True).max())
                    min_k = float(torch.norm(lipsNet.dydx, p=2, dim=1, keepdim=True).min())
                    print(f'Epoch: {epoch}, Loss:{float(loss-lipsNet.gp)}, MinCond: {min_val}, MaxK: {max_k}, MinK: {min_k}')

                    ax.clear()
                    ax.scatter(X1, X2, yy.data.numpy().reshape(X1.shape), marker= '.')
                    ax.scatter(X1, X2, yout.data.numpy().reshape(X1.shape), color='r', marker='.')

                    ax2.clear()
                    if use_sigmoid:
                        ax2.scatter(X1, X2, c=yout.data.numpy().reshape(Y.shape), s=70, edgecolors='k', lw=0.5)
                        ax2.scatter(X1, X2, c=Y, marker='.')
                    else:
                        ax2.contourf(X1, X2, yout.data.numpy().reshape(Y.shape), levels=20)

                    fig.canvas.draw()
                    plt.pause(0.01)

            if use_sigmoid:
                acc = float(((yout>0.5).type(torch.float32)==yy).type(torch.float32).mean()*100)
                info_per_seed.append(
                    f'Epoch: {epoch}, Loss:{float(loss-lipsNet.gp)}, MinCond: {min_val}, MaxK: {max_k}, MinK: {min_k}, Acc: {acc}'
                )
            else:
                info_per_seed.append(f'Epoch: {epoch}, Loss:{float(loss-lipsNet.gp)}, MinCond: {min_val}, MaxK: {max_k}, MinK: {min_k}')

                
        original_stdout = sys.stdout
        sys.stdout = open(log_file, 'a')
        print("###############################################")
        print("GC-GP")
        for i in range(len(info_per_seed)):
            print(i, network_seeds[i], info_per_seed[i])
        print(np.mean(per_step_time), np.std(per_step_time))
        print("###############################################")
        # Reset the standard output
        sys.stdout.close()
        sys.stdout = original_stdout 
        
        
        
        plt.close()
        #################################################    
        
        if lambda_ == 1:
            per_step_time = []
            info_per_seed = []

            fig = plt.figure(figsize=(15,6))
            ax = fig.add_subplot(121,projection='3d')
            ax2 = fig.add_subplot(122)
            # ax.view_init(28,20)

            for ns in network_seeds:
                torch.manual_seed(ns)

                net_lips = nn.Sequential(nn.utils.spectral_norm(nn.Linear(2,10)),
                                         actf(),
                                         nn.utils.spectral_norm(nn.Linear(10,10)),
                                         actf(),
                                         nn.utils.spectral_norm(nn.Linear(10,1)),
                                         nn.Sigmoid() if use_sigmoid else nn.Identity())

                ## no clipper and loss used..., just for getting gradient norm.
                snNet = LipschitzNet(net_lips)

                optimizer = torch.optim.Adam(snNet.parameters(), lr=learning_rate)

                for epoch in range(EPOCHS):
                    start = time.time()

                    yout = snNet(xx)
                    loss = criterion(yout, yy)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    per_step_time.append(time.time()-start)

                    if epoch%100 == 0:
                        yout = snNet(xx)
                        snNet.get_dydx()

                        max_k = float(torch.norm(snNet.dydx, p=2, dim=1, keepdim=True).max())
                        min_k = float(torch.norm(snNet.dydx, p=2, dim=1, keepdim=True).min())
                        print(f'Epoch: {epoch}, Loss:{float(loss)}, MaxK: {max_k}, MinK: {min_k}')

                        ax.clear()
                        ax.scatter(X1, X2, yy.data.numpy().reshape(X1.shape), marker= '.')
                        ax.scatter(X1, X2, yout.data.numpy().reshape(X1.shape), color='r', marker='.')

                        ax2.clear()
                        if use_sigmoid:
                            ax2.scatter(X1, X2, c=yout.data.numpy().reshape(Y.shape), s=70, edgecolors='k', lw=0.5)
                            ax2.scatter(X1, X2, c=Y, marker='.')
                        else:
                            ax2.contourf(X1, X2, yout.data.numpy().reshape(Y.shape), levels=20)

                        fig.canvas.draw()
                        plt.pause(0.01)
                        print("\n")

                if use_sigmoid:
                    acc = float(((yout>0.5).type(torch.float32)==yy).type(torch.float32).mean()*100)
                    info_per_seed.append(
                        f'Epoch: {epoch}, Loss:{float(loss)}, MaxK: {max_k}, MinK: {min_k}, Acc: {acc}'
                    )
                else:
                    info_per_seed.append(f'Epoch: {epoch}, Loss:{float(loss)}, MaxK: {max_k}, MinK: {min_k}')


            original_stdout = sys.stdout
            sys.stdout = open(log_file, 'a')
            print("###############################################")
            print("SN")
            for i in range(len(info_per_seed)):
                print(i, network_seeds[i], info_per_seed[i])
            print(np.mean(per_step_time), np.std(per_step_time))
            print("###############################################")
            # Reset the standard output
            sys.stdout.close()
            sys.stdout = original_stdout 

            plt.close()
        #################################################    
        
        per_step_time = []
        info_per_seed = []

        fig = plt.figure(figsize=(15,6))
        ax = fig.add_subplot(121,projection='3d')
        ax2 = fig.add_subplot(122)
        # ax.view_init(28,20)

        for ns in network_seeds:
            torch.manual_seed(ns)

            net_lips = nn.Sequential(nn.Linear(2,10),
                                     actf(),
                                     nn.Linear(10,10),
                                     actf(),
                                     nn.Linear(10,1),
                                     nn.Sigmoid() if use_sigmoid else nn.Identity())

            gpNet = GradientPenaltyNet(net_lips, K=1, lamda=lambda_)

            optimizer = torch.optim.Adam(gpNet.parameters(), lr=learning_rate)
            for epoch in range(EPOCHS):
                start = time.time()

                yout = gpNet(xx)    
                gpNet.compute_penalty()
                loss = criterion(yout, yy) + gpNet.gp


                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                per_step_time.append(time.time()-start)
                if epoch%100 == 0:
                    allk = torch.norm(gpNet.dydx, p=2, dim=1, keepdim=True)
                    max_k = float(allk.max())
                    min_k = float(allk.min())

                    print(f'Epoch: {epoch}, Loss:{float(loss-gpNet.gp)}, MaxK: {max_k} MinK: {min_k}')

                    ax.clear()
                    ax.scatter(X1, X2, yy.data.numpy().reshape(X1.shape), marker= '.')
                    ax.scatter(X1, X2, yout.data.numpy().reshape(X1.shape), color='r', marker='.')

                    ax2.clear()
                    if use_sigmoid:
                        ax2.scatter(X1, X2, c=yout.data.numpy().reshape(Y.shape), s=70, edgecolors='k', lw=0.5)
                        ax2.scatter(X1, X2, c=Y, marker='.')
                    else:
                        ax2.contourf(X1, X2, yout.data.numpy().reshape(Y.shape), levels=20)

                    fig.canvas.draw()
                    plt.pause(0.01)
                    print("\n")

            if use_sigmoid:
                acc = float(((yout>0.5).type(torch.float32)==yy).type(torch.float32).mean()*100)
                info_per_seed.append(
                    f'Epoch: {epoch}, Loss:{float(loss-gpNet.gp)}, MaxK: {max_k}, MinK: {min_k}, Acc: {acc}'
                )
            else:
                info_per_seed.append(f'Epoch: {epoch}, Loss:{float(loss-gpNet.gp)}, MaxK: {max_k}, MinK: {min_k}')

                
        original_stdout = sys.stdout
        sys.stdout = open(log_file, 'a')
        print("###############################################")
        print("GP")
        for i in range(len(info_per_seed)):
            print(i, network_seeds[i], info_per_seed[i])
        print(np.mean(per_step_time), np.std(per_step_time))
        print("###############################################")
        # Reset the standard output
        sys.stdout.close()
        sys.stdout = original_stdout 
        
        plt.close()
        #################################################    

        per_step_time = []
        info_per_seed = []

        fig = plt.figure(figsize=(15,6))
        ax = fig.add_subplot(121,projection='3d')
        ax2 = fig.add_subplot(122)
        # ax.view_init(28,20)

        for ns in network_seeds:
            torch.manual_seed(ns)

            net_lips = nn.Sequential(nn.Linear(2,10),
                                     actf(),
                                     nn.Linear(10,10),
                                     actf(),
                                     nn.Linear(10,1),
                                     nn.Sigmoid() if use_sigmoid else nn.Identity())

            lpNet = LipschitzPenaltyNet(net_lips, K=1, lamda=lambda_)

            optimizer = torch.optim.Adam(lpNet.parameters(), lr=learning_rate)

            for epoch in range(EPOCHS):
                start = time.time()

                yout = lpNet(xx)    
                lpNet.compute_penalty()
                loss = criterion(yout, yy) + lpNet.gp


                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                per_step_time.append(time.time()-start)

                if epoch%100 == 0:
                    allk = torch.norm(lpNet.dydx, p=2, dim=1, keepdim=True)
                    max_k = float(allk.max())
                    min_k = float(allk.min())

                    print(f'Epoch: {epoch}, Loss:{float(loss-lpNet.gp)}, MaxK: {max_k} MinK: {min_k}')

                    ax.clear()
                    ax.scatter(X1, X2, yy.data.numpy().reshape(X1.shape), marker= '.')
                    ax.scatter(X1, X2, yout.data.numpy().reshape(X1.shape), color='r', marker='.')

                    ax2.clear()
                    if use_sigmoid:
                        ax2.scatter(X1, X2, c=yout.data.numpy().reshape(Y.shape), s=70, edgecolors='k', lw=0.5)
                        ax2.scatter(X1, X2, c=Y, marker='.')
                    else:
                        ax2.contourf(X1, X2, yout.data.numpy().reshape(Y.shape), levels=20)

                    fig.canvas.draw()
                    plt.pause(0.01)
                    print("\n")
            if use_sigmoid:
                acc = float(((yout>0.5).type(torch.float32)==yy).type(torch.float32).mean()*100)
                info_per_seed.append(
                    f'Epoch: {epoch}, Loss:{float(loss-lpNet.gp)}, MaxK: {max_k}, MinK: {min_k}, Acc: {acc}'
                )
            else:
                info_per_seed.append(f'Epoch: {epoch}, Loss:{float(loss-lpNet.gp)}, MaxK: {max_k}, MinK: {min_k}')

                
        original_stdout = sys.stdout
        sys.stdout = open(log_file, 'a')
        print("###############################################")
        print("LP")
        for i in range(len(info_per_seed)):
            print(i, network_seeds[i], info_per_seed[i])
        print(np.mean(per_step_time), np.std(per_step_time))
        print("###############################################")
        # Reset the standard output
        sys.stdout.close()
        sys.stdout = original_stdout 
        
        plt.close()

        #################################################    
                

Epoch: 0, Loss:0.4675413966178894, MinCond: 0.8093889951705933, MaxK: 0.19061100482940674, MinK: 0.09312445670366287
Epoch: 100, Loss:0.1101626455783844, MinCond: 0.06541401147842407, MaxK: 0.9345859885215759, MinK: 0.32371005415916443
Epoch: 200, Loss:0.10048636794090271, MinCond: 0.060855329036712646, MaxK: 0.9391446709632874, MinK: 0.08893952518701553
Epoch: 300, Loss:0.09841816872358322, MinCond: 0.06962788105010986, MaxK: 0.9303721189498901, MinK: 0.003657673019915819
Epoch: 400, Loss:0.09749960154294968, MinCond: 0.062367141246795654, MaxK: 0.9376328587532043, MinK: 0.05337119847536087
Epoch: 500, Loss:0.09665811061859131, MinCond: 0.04284536838531494, MaxK: 0.9571546316146851, MinK: 0.08605284988880157
Epoch: 600, Loss:0.09575740993022919, MinCond: 0.041986703872680664, MaxK: 0.9580132961273193, MinK: 0.024017874151468277
Epoch: 700, Loss:0.09485293924808502, MinCond: 0.0292627215385437, MaxK: 0.9707372784614563, MinK: 0.005432970356196165
Epoch: 800, Loss:0.0937163308262825, Mi

Epoch: 6800, Loss:0.08240922540426254, MinCond: 0.07991290092468262, MaxK: 0.9200870990753174, MinK: 0.012734458781778812
Epoch: 6900, Loss:0.08340948820114136, MinCond: 0.07596248388290405, MaxK: 0.924037516117096, MinK: 0.01448710635304451
Epoch: 7000, Loss:0.08375763893127441, MinCond: 0.07110750675201416, MaxK: 0.9288924932479858, MinK: 0.011840272694826126
Epoch: 7100, Loss:0.0828489288687706, MinCond: 0.0769193172454834, MaxK: 0.9230806827545166, MinK: 0.012198586016893387
Epoch: 7200, Loss:0.08240754157304764, MinCond: 0.08247685432434082, MaxK: 0.9175231456756592, MinK: 0.00430163973942399
Epoch: 7300, Loss:0.08296964317560196, MinCond: 0.07838153839111328, MaxK: 0.9216184616088867, MinK: 0.00330479652620852
Epoch: 7400, Loss:0.0828145295381546, MinCond: 0.08073240518569946, MaxK: 0.9192675948143005, MinK: 0.008122405968606472
Epoch: 0, Loss:0.46714794635772705, MinCond: 0.9141939282417297, MaxK: 0.08580604940652847, MinK: 0.04806589335203171
Epoch: 100, Loss:0.109778493642807,

Epoch: 6100, Loss:0.08353894203901291, MinCond: 0.02047741413116455, MaxK: 0.9795225858688354, MinK: 0.007050294429063797
Epoch: 6200, Loss:0.08360236883163452, MinCond: 0.02302253246307373, MaxK: 0.9769774675369263, MinK: 0.00961578544229269
Epoch: 6300, Loss:0.08531726151704788, MinCond: 0.015578687191009521, MaxK: 0.9844213128089905, MinK: 0.004902179352939129
Epoch: 6400, Loss:0.08574827760457993, MinCond: 0.01796698570251465, MaxK: 0.9820330142974854, MinK: 0.014936643652617931
Epoch: 6500, Loss:0.08367084711790085, MinCond: 0.022210240364074707, MaxK: 0.9777897596359253, MinK: 0.003326146164909005
Epoch: 6600, Loss:0.08337639272212982, MinCond: 0.02182769775390625, MaxK: 0.9781723022460938, MinK: 0.004339209292083979
Epoch: 6700, Loss:0.08335385471582413, MinCond: 0.022084057331085205, MaxK: 0.9779159426689148, MinK: 0.0073048812337219715
Epoch: 6800, Loss:0.08558221161365509, MinCond: 0.01958400011062622, MaxK: 0.9804159998893738, MinK: 0.023269852623343468
Epoch: 6900, Loss:0.0

Epoch: 5400, Loss:0.08401594310998917, MinCond: 0.06106007099151611, MaxK: 0.9389399290084839, MinK: 0.02743564359843731
Epoch: 5500, Loss:0.08266556262969971, MinCond: 0.0668715238571167, MaxK: 0.9331284761428833, MinK: 0.014935730025172234
Epoch: 5600, Loss:0.08365947008132935, MinCond: 0.062144696712493896, MaxK: 0.9378553032875061, MinK: 0.026720372959971428
Epoch: 5700, Loss:0.08265198767185211, MinCond: 0.06637781858444214, MaxK: 0.9336221814155579, MinK: 0.01639643684029579
Epoch: 5800, Loss:0.08491005003452301, MinCond: 0.057819366455078125, MaxK: 0.9421806335449219, MinK: 0.030530771240592003
Epoch: 5900, Loss:0.08421174436807632, MinCond: 0.06021994352340698, MaxK: 0.939780056476593, MinK: 0.02863357774913311
Epoch: 6000, Loss:0.08413416892290115, MinCond: 0.06039971113204956, MaxK: 0.9396002888679504, MinK: 0.028615547344088554
Epoch: 6100, Loss:0.0828285738825798, MinCond: 0.06583744287490845, MaxK: 0.9341625571250916, MinK: 0.021839113906025887
Epoch: 6200, Loss:0.08219843

Epoch: 6200, Loss:0.0915423184633255, MaxK: 0.9967157244682312, MinK: 0.12129813432693481


Epoch: 6300, Loss:0.09212289750576019, MaxK: 0.9887887835502625, MinK: 0.12394104152917862


Epoch: 6400, Loss:0.09157013893127441, MaxK: 0.9995726943016052, MinK: 0.11784820258617401


Epoch: 6500, Loss:0.09176874905824661, MaxK: 0.9887830018997192, MinK: 0.12391801923513412


Epoch: 6600, Loss:0.09272649884223938, MaxK: 0.9802227020263672, MinK: 0.12854284048080444


Epoch: 6700, Loss:0.09115777909755707, MaxK: 0.9994391202926636, MinK: 0.12281535565853119


Epoch: 6800, Loss:0.09361371397972107, MaxK: 0.9672448635101318, MinK: 0.1276865452528


Epoch: 6900, Loss:0.09137604385614395, MaxK: 0.999270498752594, MinK: 0.12278765439987183


Epoch: 7000, Loss:0.09094145148992538, MaxK: 0.9985159635543823, MinK: 0.12396927922964096


Epoch: 7100, Loss:0.09220604598522186, MaxK: 0.9994620084762573, MinK: 0.12149429321289062


Epoch: 7200, Loss:0.09162447601556778, MaxK: 0.9987316727638245, MinK: 0.123

...

Epoch: 3600, Loss:0.06275954097509384, MaxK: 1.1195846796035767 MinK: 0.026772765442728996


Epoch: 3700, Loss:0.06422679871320724, MaxK: 1.1421928405761719 MinK: 0.027383413165807724


Epoch: 3800, Loss:0.06295429170131683, MaxK: 1.1268649101257324 MinK: 0.030282657593488693


Epoch: 3900, Loss:0.063108429312706, MaxK: 1.1286540031433105 MinK: 0.03235786035656929


Epoch: 4000, Loss:0.06408804655075073, MaxK: 1.1391031742095947 MinK: 0.03667834401130676


Epoch: 4100, Loss:0.06435959786176682, MaxK: 1.1418458223342896 MinK: 0.03743693605065346


Epoch: 4200, Loss:0.06407523155212402, MaxK: 1.1461690664291382 MinK: 0.029402416199445724


Epoch: 4300, Loss:0.06314340978860855, MaxK: 1.1262462139129639 MinK: 0.011826497502624989


Epoch: 4400, Loss:0.06351687014102936, MaxK: 1.140488862991333 MinK: 0.010601851157844067


Epoch: 4500, Loss:0.06298745423555374, MaxK: 1.1355100870132446 MinK: 0.0073956991545856


Epoch: 4600, Loss:0.06393514573574066, MaxK: 1.144128441810608 MinK: 0.0056177

Epoch: 5100, Loss:0.06387538462877274, MaxK: 1.1441816091537476 MinK: 0.03393784537911415


Epoch: 5200, Loss:0.06447946280241013, MaxK: 1.1523550748825073 MinK: 0.03923199698328972


Epoch: 5300, Loss:0.06336845457553864, MaxK: 1.1325312852859497 MinK: 0.03017386421561241


Epoch: 5400, Loss:0.06433518975973129, MaxK: 1.1498913764953613 MinK: 0.04133095592260361


Epoch: 5500, Loss:0.06390949338674545, MaxK: 1.1420162916183472 MinK: 0.04140039160847664


Epoch: 5600, Loss:0.06468665599822998, MaxK: 1.1526873111724854 MinK: 0.04170599952340126


Epoch: 5700, Loss:0.06438612192869186, MaxK: 1.1474031209945679 MinK: 0.044963326305150986


Epoch: 5800, Loss:0.06393054872751236, MaxK: 1.1388463973999023 MinK: 0.045341700315475464


Epoch: 5900, Loss:0.06400755792856216, MaxK: 1.1406809091567993 MinK: 0.04213852807879448


Epoch: 6000, Loss:0.06287841498851776, MaxK: 1.121561050415039 MinK: 0.03267575427889824


Epoch: 6100, Loss:0.06429096311330795, MaxK: 1.1437901258468628 MinK: 0.0433258

Epoch: 6600, Loss:0.06286322325468063, MaxK: 1.126990556716919 MinK: 0.016843637451529503


Epoch: 6700, Loss:0.0637902319431305, MaxK: 1.1434717178344727 MinK: 0.014691635966300964


Epoch: 6800, Loss:0.06396042555570602, MaxK: 1.154653787612915 MinK: 0.007477167062461376


Epoch: 6900, Loss:0.06300686299800873, MaxK: 1.1512763500213623 MinK: 0.007832604460418224


Epoch: 7000, Loss:0.06324641406536102, MaxK: 1.162447214126587 MinK: 0.01505741011351347


Epoch: 7100, Loss:0.06316426396369934, MaxK: 1.19184410572052 MinK: 0.02197723649442196


Epoch: 7200, Loss:0.0634208470582962, MaxK: 1.1986216306686401 MinK: 0.02646363340318203


Epoch: 7300, Loss:0.06426677107810974, MaxK: 1.2237181663513184 MinK: 0.030960995703935623


Epoch: 7400, Loss:0.06293823570013046, MaxK: 1.2071939706802368 MinK: 0.0341397300362587


Epoch: 0, Loss:0.23010878264904022, MinCond: 0.8095769882202148, MaxK: 0.19042302668094635, MinK: 0.09312353283166885
Epoch: 100, Loss:0.030157597735524178, MinCond: 0.7691374

Epoch: 6100, Loss:0.007284392137080431, MinCond: 0.09523904323577881, MaxK: 0.9047609567642212, MinK: 0.026474904268980026
Epoch: 6200, Loss:0.007266454864293337, MinCond: 0.10394018888473511, MaxK: 0.8960598111152649, MinK: 0.014903444796800613
Epoch: 6300, Loss:0.0073552182875573635, MinCond: 0.08526778221130371, MaxK: 0.9147322177886963, MinK: 0.028686530888080597
Epoch: 6400, Loss:0.00725167989730835, MinCond: 0.10490643978118896, MaxK: 0.895093560218811, MinK: 0.01161852478981018
Epoch: 6500, Loss:0.0072674741968512535, MinCond: 0.10466516017913818, MaxK: 0.8953348398208618, MinK: 0.00736223952844739



KeyboardInterrupt



In [ ]:
# break the code